In [ ]:
#test GloVe model with same word, one caps, other not caps

In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import re

In [2]:
import os
import shutil
import smart_open
from sys import platform
import gensim

In [3]:
def prepend_line(infile, outfile, line):
    """ 
    Function use to prepend lines using bash utilities in Linux. 
    (source: http://stackoverflow.com/a/10850588/610569)
    """
    with open(infile, 'r', encoding="utf8") as old:
        with open(outfile, 'w', encoding="utf8") as new:
            new.write(str(line) + "\n")
            shutil.copyfileobj(old, new)

def prepend_slow(infile, outfile, line):
    """
    Slower way to prepend the line by re-creating the inputfile.
    """
    with open(infile, 'r', encoding="utf8") as fin:
        with open(outfile, 'w', encoding="utf8") as fout:
            fout.write(line + "\n")
            for line in fin:
                fout.write(line)

def get_lines(glove_file_name):
    """Return the number of vectors and dimensions in a file in GloVe format."""
    with smart_open.smart_open(glove_file_name, 'r', encoding="utf8") as f:
        num_lines = sum(1 for line in f)
    with smart_open.smart_open(glove_file_name, 'r', encoding="utf8") as f:
        num_dims = len(f.readline().split()) - 1
    return num_lines, num_dims

# Input: GloVe Model File
# More models can be downloaded from http://nlp.stanford.edu/projects/glove/
#glove_file="glove.6B.300d.txt"
glove_file= r'C:\Users\nikij\Desktop\college\capstone\vaccine\Symptom Prediction\glove.42B.300d\glove.42B.300d.txt'

num_lines, dims = get_lines(glove_file)

# Output: Gensim Model text format.
gensim_file='glove_model2.txt'
gensim_first_line = "{} {}".format(num_lines, dims)

# Prepends the line.
if platform == "linux" or platform == "linux2":
    prepend_line(glove_file, gensim_file, gensim_first_line)
else:
    prepend_slow(glove_file, gensim_file, gensim_first_line)

# Demo: Loads the newly created glove_model.txt into gensim API.
model=gensim.models.KeyedVectors.load_word2vec_format(gensim_file,binary=False) #GloVe Model

In [9]:
l = ['Para','Cata']

for i in l:
    print(i.lower())
print(model['Paracetamol'.lower()])

para
cata
[ 0.057777  -0.065165   0.46952    0.074488   0.45392   -0.40041
 -0.39756    0.35879    0.32403   -0.51244    0.10301    0.24029
 -0.74578    0.10445   -0.63752    0.47939    0.51449    0.58795
 -0.68408    0.31802    0.076313  -0.29599   -0.38381   -0.062685
  0.48119   -0.41661    0.52723   -0.80899    0.30443   -0.022983
  0.090743  -0.35872   -0.11547   -0.54067    0.32223   -0.31708
 -0.26967   -0.58827    0.030474  -0.42215   -0.6473    -0.24239
 -0.66428   -0.71383   -0.056201   0.13471    0.17543    0.2095
  0.28686   -0.20854   -0.36009    0.5571     0.031856  -0.29795
 -0.022248   0.37339    0.010434   0.017707  -0.06727   -0.69088
 -0.5294    -0.29336   -0.74367    0.16231   -0.32833    0.3725
  0.60718    0.0039399 -0.40087   -0.59991    0.089434   0.35092
  0.64336    0.53996    0.6468     0.16951    0.26801   -0.05911
 -0.74138    0.13343    0.25689   -0.058544   0.22232   -0.022667
  0.37799    0.43947   -0.39901    0.53077   -0.83845    0.49801
  0.55239   -0

In [10]:
exp = "[a-zA-Z0-9]+"

In [11]:
def vector_avg(vec_list):
        return sum(vec_list)/len(vec_list)

    #return np.mean(vec_list)


def get_vector(feature):   #"kirk vit"
    split_feature=feature.split()   #["kirk","vit"]
    
    vec_list=[]
    for item in feature:
        try:
            vec_list.append(model[item.lower()])  #converting each word in the medicine name to a vector
        except:
            #print("exception!!")
            continue
        
        
        '''if model[item]:
            vec_list.append(model[item])  #converting each word in the medicine name to a 
        else:
            continue'''
        
    #calc vector avg of all words in medicine name    
    vector=vector_avg(vec_list)    
        
    return vector
    
    
def get_similarity(vector):
    sim=model.similar_by_vector(vector, topn=11, restrict_vocab=None)
    return sim
    

In [12]:
def string_to_vector(feature_str):
    feats=re.findall(exp,feature_str)
    #print(meds)    #['kirkland multivitamin', 'kirkland calcium vitamin', 'vitamin d', 'fish oil']

    #list of vectors of the medicine names in each row
    #[v(kirkland multivitamin), v(kirkland calcium vitamin), v(vitamin d), v(fish oil)]
    vec_list=[]  

    for item in feats:
        #try:             
        v=get_vector(item)
        vec_list.append(v)                            

    #avg of all med vectors in a row (for a user/entry)
    if vec_list:
        feats_avg=vector_avg(vec_list)
        return feats_avg
    else:
        return -1
#        return np.empty(300,)

In [13]:
def add_vector_column(df):
    df["vector"] = ""
    for i in range(len(df)):
        feature_count=0
        res_vect = np.empty(300,)
        if not pd.isnull(df['other_meds_filtered'][i]):
            vector = string_to_vector(df['other_meds_filtered'][i])
            if type(vector) != int:
                res_vect += vector
                feature_count+=1
        if not pd.isnull(df['allergies_filtered'][i]):
            vector = string_to_vector(df['allergies_filtered'][i])
            if type(vector) != int:
                res_vect += vector
                feature_count+=1
        if not pd.isnull(df['history_filtered'][i]):
            vector = string_to_vector(df['history_filtered'][i])
            if type(vector) != int:
                res_vect += vector
                feature_count+=1
        if feature_count > 0:
            final_vect = res_vect/feature_count
            df["vector"].loc[i]= final_vect
            

In [ ]:
def add_med_vector_column(df): 
    df["meds_vector"] = ""

    for i,j in df["other_meds_filtered"].iteritems():
        #j = "['kirkland multivitamin', 'kirkland calcium vitamin', 'vitamin d', 'fish oil']"

        #if not pd.isna(j):
        if not pd.isnull(j):
            meds_avg = string_to_vector(j)
            if type(meds_avg) != int:
                df["meds_vector"].loc[i]= meds_avg

In [ ]:
def add_allergy_vector_column(df): 
    df["allergy_vector"] = ""

    for i,j in df["allergies_filtered"].iteritems():
        #j = "['kirkland multivitamin', 'kirkland calcium vitamin', 'vitamin d', 'fish oil']"

        #if not pd.isna(j):
        if not pd.isnull(j):
            allergies_avg = string_to_vector(j)
            if type(allergies_avg) != int:
                df["allergy_vector"].loc[i]= allergies_avg

In [ ]:
def add_hoi_vector_column(df): 
    df["hoi_vector"] = ""

    for i,j in df["history_filtered"].iteritems():
        #j = "['kirkland multivitamin', 'kirkland calcium vitamin', 'vitamin d', 'fish oil']"

        #if not pd.isna(j):
        if not pd.isnull(j):
            hoi_avg = string_to_vector(j)
            #if vect not in vect form
            if type(hoi_avg) != int:
                df["hoi_vector"].loc[i]= hoi_avg

In [14]:
data = pd.read_csv(r'C:\Users\nikij\Desktop\college\capstone\vaccine\Data\2021cleaned\filtered_cols.csv')
add_vector_column(data)
'''
add_med_vector_column(data)
add_allergy_vector_column(data)
add_hoi_vector_column(data)
'''
#testing

C:\Users\nikij\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (7,12,15,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\nikij\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


'\nadd_med_vector_column(data)\nadd_allergy_vector_column(data)\nadd_hoi_vector_column(data)\n'

In [16]:
#data_pre_finvect = data
data_pre_finvect.head()

NameError: name 'data_pre_finvect' is not defined

In [15]:
data.head()

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES,other_meds_filtered,allergies_filtered,history_filtered,vector
0,916600,01-01-2021,TX,33.0,33.0,NaN,F,NaN,Right side of epiglottis swelled up and hinder...,NaN,...,2,01-01-2021,NaN,Y,NaN,Pcn and bee venom,NaN,"['pcn', 'bee venom']",NaN,"[0.1735476702451706, 0.27377817034721375, 0.10..."
1,916601,01-01-2021,CA,73.0,73.0,NaN,F,NaN,Approximately 30 min post vaccination administ...,NaN,...,2,01-01-2021,NaN,Y,NaN,"""Dairy""",['residing nursing facility . patients chart .'],"[""`` dairy ''""]",['residing nursing facility . patients chart .'],"[0.16078740855058035, 0.32525471846262616, -0...."
2,916602,01-01-2021,WA,23.0,23.0,NaN,F,NaN,"About 15 minutes after receiving the vaccine, ...",NaN,...,2,01-01-2021,NaN,NaN,Y,Shellfish,NaN,['shellfish'],NaN,"[0.4121169000864029, 0.4279550611972809, 0.266..."
3,916603,01-01-2021,WA,58.0,58.0,NaN,F,NaN,"extreme fatigue, dizziness,. could not lift my...",NaN,...,2,01-01-2021,NaN,NaN,NaN,"Diclofenac, novacaine, lidocaine, pickles, tom...",NaN,"['diclofenac', 'novacaine', 'lidocaine', 'pick...","['diverticulitis', 'mitral valve prolapse', 'o...","[0.22408753881851834, 0.40486509849627816, 0.0..."
4,916604,01-01-2021,TX,47.0,47.0,NaN,F,NaN,"Injection site swelling, redness, warm to the ...",NaN,...,2,01-01-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [ ]:
x = input()
if not x:
    print("haha")

In [ ]:
x = []
x = np.empty(1,)
x

In [ ]:
'''
print(type(data['meds_vector'][0]))
print(type(data['meds_vector'][1]) == np.ndarray)
print(data['meds_vector'][0])
print(data['meds_vector'][0] == '')
print(data['meds_vector'][1].shape)
print(data['allergy_vector'][1].shape)
print(data['hoi_vector'][1].shape)
x = np.empty(300,)
print(x + data['hoi_vector'][1])
'''

#print(data['meds_vector'][1] + data['meds_vector'][0])
#x = data['allergy_vector'][0] + data['allergy_vector'][1]
#x

In [17]:
sympt_df = pd.read_csv(r'C:\Users\nikij\Desktop\college\capstone\vaccine\Data\2021cleaned\2021VAERSSYMPTOMS_clean.csv')

vax_df = pd.read_csv(r'C:\Users\nikij\Desktop\college\capstone\vaccine\Data\2021cleaned\2021VAERSVAX_clean.csv')
janssen_df = vax_df[vax_df['VAX_NAME'] == 'COVID19 (COVID19 (JANSSEN))']
moderna_df = vax_df[vax_df['VAX_NAME'] == 'COVID19 (COVID19 (MODERNA))']
pfizer_df = vax_df[vax_df['VAX_NAME'] == 'COVID19 (COVID19 (PFIZER-BIONTECH))']

C:\Users\nikij\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [18]:
#def get_symptoms(vax_df,age_grp_df,user_medvec,user_allervec,user_hoivec):
def get_symptoms(vax_df,age_grp_df,user_vec):
    age_vax_df = pd.merge(vax_df, age_grp_df, how = 'inner', on = 'VAERS_ID')
    #age_vax_df = age_vax_df[['VAERS_ID','VAX_NAME','meds_vector']]
    age_vax_df = age_vax_df[['VAERS_ID','VAX_NAME','vector']]
    
    #print(len(age_vax_df))
    #print(age_vax_df.head())
    #exit()
    
    age_vax_df['similarity_score']=""
    #for i,j in age_vax_df["meds_vector"].iteritems():
    for i,j in age_vax_df["vector"].iteritems():
        if type(j) == np.ndarray:
            #print(type(j))
            '''
            meds_sim_score = np.dot(user_medvec, j)/(np.linalg.norm(user_medvec)* np.linalg.norm(j))
            allergy_sim_score = np.dot(user_allervec, j)/(np.linalg.norm(user_allervec)* np.linalg.norm(j))
            hoi_sim_score = np.dot(user_hoivec, j)/(np.linalg.norm(user_hoivec)* np.linalg.norm(j))
            #avg of 3 vectors
            sim_score = (meds_sim_score + allergy_sim_score + hoi_sim_score)/3
            '''
            sim_score = np.dot(user_vec, j)/(np.linalg.norm(user_vec)* np.linalg.norm(j))
            #age_vax_df["similarity_score"].loc[i]= sim_score
            age_vax_df.loc[i, "similarity_score"]= sim_score
            #print(i,type(sim_score))
        else:
            #print(type(sim_score))
            type(j)
            #age_vax_df["similarity_score"].loc[i]=None
            age_vax_df.loc[i, "similarity_score"]= None

    #age_vax_df.sort_values(by=['similarity_score'], ascending=False)
    #print(age_vax_df.head())
    #exit()

    #sympt_match_df = pd.merge(age_vax_df, sympt_df, how = 'left', on = 'VAERS_ID')
    sympt_match_df = pd.merge(age_vax_df, sympt_df, how = 'inner', on = 'VAERS_ID')
    sympt_match_df = sympt_match_df[['VAERS_ID','vector','similarity_score','SYMPTOM1','SYMPTOM2','SYMPTOM3','SYMPTOM4','SYMPTOM5']]
    #sympt_match_df = sympt_match_df[sympt_match_df['similarity_score'].notna()]
    
    #AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
    #sympt_match_df = sympt_match_df.sort_values(by=['similarity_score'], ascending=False)
    sympt_match_df.sort_values(by=['similarity_score'], ascending=False, inplace=True)
    sympt_match_df = sympt_match_df.reset_index(drop=True)
    #print(sympt_match_df.head())
    #return []


    pred_symptoms = []
    for i in range(len(sympt_match_df)):
        if len(pred_symptoms) >=5:
            break
        
        if not (pd.isnull(sympt_match_df.at[i,'SYMPTOM1'])) and (sympt_match_df['SYMPTOM1'][i] not in pred_symptoms):
            pred_symptoms.append(sympt_match_df['SYMPTOM1'][i])   
        if len(pred_symptoms) >=5:
            break
        if not (pd.isnull(sympt_match_df.at[i,'SYMPTOM2'])) and sympt_match_df['SYMPTOM2'][i] not in pred_symptoms:
            pred_symptoms.append(sympt_match_df['SYMPTOM2'][i])
        if len(pred_symptoms) >=5:
            break
        if not (pd.isnull(sympt_match_df.at[i,'SYMPTOM3'])) and sympt_match_df['SYMPTOM3'][i] not in pred_symptoms:
            pred_symptoms.append(sympt_match_df['SYMPTOM3'][i])
        if len(pred_symptoms) >=5:
            break
        if not (pd.isnull(sympt_match_df.at[i,'SYMPTOM4'])) and sympt_match_df['SYMPTOM4'][i] not in pred_symptoms:
            pred_symptoms.append(sympt_match_df['SYMPTOM4'][i])
        if len(pred_symptoms) >=5:
            break
        if not (pd.isnull(sympt_match_df.at[i,'SYMPTOM5'])) and sympt_match_df['SYMPTOM5'][i] not in pred_symptoms:
            pred_symptoms.append(sympt_match_df['SYMPTOM5'][i])

    return pred_symptoms

In [ ]:
#On the basis of age group and medications taken, 
#similarity between users is found

#Assumptions
#Meds is a compulsory field; not empty
#Meds selected from a searchable drop down menu, 
#so it will be an exact match to what's been used as sample input

In [19]:
def run_model():
    try:
        age = int(input("Enter age: "))
    except:
        print("Invalid age input. Please try again.")
        return
        #exit()

    feat_count = 0
    med_entry = input("Enter list of medications:\n")
    if not med_entry:
        print("Invalid input. Please enter medications")
        return
    feat_count+=1
    allergy_entry = input("Enter list of allergies:\n")
    if allergy_entry:
        feat_count+=1
    hoi_entry = input("Enter list of prior illness:\n")
    if hoi_entry:
        feat_count+=1

    age_grp_df = data.loc[(data['AGE_YRS'] >= age-5) & (data['AGE_YRS'] <= age+5)]
    #CONVERT INPUT MEDS INTO VECTOR
    user_med_vector=string_to_vector(med_entry)
    user_allergy_vector=string_to_vector(allergy_entry)
    user_hoi_vector=string_to_vector(hoi_entry)
    user_vector=(user_med_vector+user_allergy_vector+user_hoi_vector)/3
    #print(user_med_vector)
    #return

    #COVID19 (COVID19 (JANSSEN)) #COVID19 (COVID19 (MODERNA)) #COVID19 (COVID19 (PFIZER-BIONTECH))
    vax_list = ['COVID19 (COVID19 (MODERNA))','COVID19 (COVID19 (PFIZER-BIONTECH))'] 
    
    #print(len(moderna_df))
    #print(moderna_df.head())
    #return
    print("\nThe available vaccines are:")
    for vax in vax_list:
        symptom_set = []
        print("\n**********************************************************")
        print("VACCINE NAME:",vax[18:-2])
        if vax == 'COVID19 (COVID19 (MODERNA))':
            symptom_set = get_symptoms(moderna_df,age_grp_df,user_vector)
            #risk = 1.83
        elif vax == 'COVID19 (COVID19 (PFIZER-BIONTECH))':
            symptom_set = get_symptoms(pfizer_df,age_grp_df,user_vector)
            #risk = 2.67
        #print("Risk Percentage: ",risk,"%\n",sep="")
        print("POTENTIAL SYMPTOMS (in order of likelihood):")
        for i in range(len(symptom_set)):
            print(i+1,". ",symptom_set[i],sep="")
        print("**********************************************************\n")


In [ ]:
#user examples
#40 #50 #71 #35

#kirkland multivitamin, kirkland calcium vitamin, vitamin d, fish oil
#amlodipine, ambien, benicar/hctz, invokana, metformin, levothyroxine, bydureon, metoprolol
#levothyroxine, estradiol
#fluoxetine qd, cetirizine qd apple cider vinegar pills

In [20]:
run_model()

Enter age: 50
Enter list of medications:
Amlodipine, Ambien, Benicar/HCTZ, Invokana, Metformin, Levothyroxine, Bydureon, Metoprolol
Enter list of allergies:
penicillin
Enter list of prior illness:
High blood pressure, high cholesterol, sleep apnea, insomnia, diabetes type II, obesity

The available vaccines are:

**********************************************************
VACCINE NAME: MODERNA
POTENTIAL SYMPTOMS (in order of likelihood):
1. Abdominal pain
2. Chills
3. Sleep disorder
4. Heart rate increased
5. Pain
**********************************************************


**********************************************************
VACCINE NAME: PFIZER-BIONTECH
POTENTIAL SYMPTOMS (in order of likelihood):
1. Dysmenorrhoea
2. Hypomenorrhoea
3. Laboratory test normal
4. Menopause
5. Menstruation irregular
**********************************************************



In [ ]:
run_model()

In [ ]:
run_model()

In [ ]:
run_model()

In [ ]:
run_model()

In [ ]:
run_model()

In [ ]:
run_model()

In [ ]:
#rank the symptoms according to likelihood

#Color code the more likely symptoms -> will be improving the UI look for the Dec review. 
#For now, we are still determining the appropriate thresholds for color coding

In [ ]:
Give use case, explain the dataset
Young male, elder female, etc., diff med histories
Specify that it is real-time data, can be streamed live (future scope)
Highlight the pretrained model while explaining